In [ ]:
# pip install mysql-connector-python
# to install mysql connector 

In [ ]:
import pandas as pd
from tabulate import tabulate

In [ ]:
data = pd.read_excel("C:/Users/Deepanshu/DA/Datasets/ESD.xlsx")

In [ ]:
data.info()

In [ ]:
data.duplicated('EEID').sum()

In [ ]:
# to find if there are any duplicate record for the same employee. 
# Since there can be multiple employees with same name or age or country or ethnicity, 
# I have considered multiple parameters to check there are duplicate records for the same employees
data[data.duplicated(['Full Name', 'Age', 'Ethnicity', 'Country'])]

In [ ]:
# There are no duplicate records for the same employee, however there are duplicate EEID numbers. 
# Hence We have to drop the 'EEID' and add a new column as the EEID with unique values. 
data.drop(['EEID'], axis=1, inplace=True)
# Generate Unique employee ids
unique_eeids = range(1, len(data)+1)
data['EEID'] = unique_eeids

In [ ]:
# rearranging the columns in the dataframe
data = data[['EEID', 'Full Name', 'Job Title', 'Department', 'Business Unit', 'Gender',
       'Ethnicity', 'Age', 'Hire Date', 'Annual Salary', 'Bonus %', 'Country',
       'City', 'Exit Date']]

### Connecting to a database.

In [ ]:
import mysql.connector

In [ ]:
conn = mysql.connector.connect(
    host = 'localhost', 
    user = 'root', 
    password = 'root123', 
    database = 'db1'
)

if conn.is_connected():
    print("Connection established !")

### Creating a table

In [ ]:
cur = conn.cursor()

In [ ]:
query = '''
create table employees (
    EEID varchar(5) not null primary key,
    `Full Name` varchar(255) not null,
    `Job Title` varchar(255) not null,
    Department varchar(255) not null, 
    `Business Unit` VARCHAR(255) NOT NULL,
    Gender VARCHAR(10) NOT NULL,
    Ethnicity VARCHAR(50) NOT NULL,
    Age INT NOT NULL,
    `Hire Date` DATE NOT NULL,
    `Annual Salary` DECIMAL(10, 2) NOT NULL,
    `Bonus %` DECIMAL(5, 2) NOT NULL,
    Country VARCHAR(50) NOT NULL,
    City VARCHAR(50) NOT NULL,
    `Exit Date` DATE
);
'''

In [ ]:
cur.execute(query)

### Inserting records into table

In [ ]:
query = '''
insert into employees (EEID, `Full Name`, `Job Title`, Department, `Business Unit`, Gender, Ethnicity, Age, `Hire Date`, `Annual Salary`, `Bonus %`, Country, City, `Exit Date`) 
values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

In [ ]:
for index, row in data.iterrows(): 
    cur.execute(query, (row['EEID'], row['Full Name'], row['Job Title'], row['Department'], row['Business Unit'], row['Gender'], row['Ethnicity'], row['Age'], row['Hire Date'], row['Annual Salary'], row['Bonus %'], row['Country'], row['City'], row['Exit Date']))

In [ ]:
conn.commit()

In [ ]:
for index, row in data.iterrows(): 
    print((row['EEID'], row['Full Name'], row['Job Title'], row['Department'], row['Business Unit'], row['Gender'], row['Ethnicity'], row['Age'], row['Hire Date'], row['Annual Salary'], row['Bonus %'], row['Country'], row['City'], row['Exit Date']))

### Fetching data

In [ ]:
query = '''
select * from employees
'''

In [ ]:
cur.execute(query)

In [ ]:
result = cur.fetchall()

In [ ]:
for record in result:
    print(record)

# Exploratory Data Analysis using SQL

##### Q. How many employees are represented in the database? 

In [16]:
query = '''
select count(EEID) from employees;
'''

In [17]:
cur.execute(query)

In [18]:
result = cur.fetchall()

In [19]:
print(result)

[(1000,)]


##### Q. What is the distribution of genders across the employees? 

In [20]:
query = '''
select 
	gender as Gender, 
    count(EEID) as Gender_Count
from db1.employees
group by gender;
'''

In [21]:
cur.execute(query)

In [22]:
result = cur.fetchall()

In [23]:
print(result)

[('Female', 518), ('Male', 482)]


In [24]:
# python function display result in grid format
def print_result(result):
    # get the column names
    column_names = [column[0] for column in cur.description]

    # print the result as table
    print(tabulate(result, headers=column_names, tablefmt='grid'))

In [25]:
print_result(result)

+----------+----------------+
| Gender   |   Gender_Count |
+==========+================+
| Female   |            518 |
+----------+----------------+
| Male     |            482 |
+----------+----------------+


##### Q. What are the common job titles within the company?

In [26]:
query = '''
select 
    distinct `job title` as `Job Title`, 
    count(EEID) over (partition by `Job Title`) as `Count of Job Title`
from employees 
order by `Count of Job Title` desc
limit 0, 5;
'''

In [27]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+----------------+----------------------+
| Job Title      |   Count of Job Title |
+================+======================+
| Director       |                  121 |
+----------------+----------------------+
| Sr. Manger     |                  110 |
+----------------+----------------------+
| Vice President |                  105 |
+----------------+----------------------+
| Manager        |                   98 |
+----------------+----------------------+
| Sr. Analyst    |                   70 |
+----------------+----------------------+


##### Q. What is the average age of employees within the company? 

In [28]:
query = '''
select avg(age) from employees; 
'''

In [29]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+------------+
|   avg(age) |
+============+
|     44.382 |
+------------+


##### Q. What is the distribution of enthnicities among the employees?

In [30]:
query = '''
select 
    distinct Ethnicity, 
    Gender, 
    count(EEID) over (partition by Ethnicity, Gender) as Count
from employees;
'''

In [31]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-------------+----------+---------+
| Ethnicity   | Gender   |   Count |
+=============+==========+=========+
| Asian       | Female   |     207 |
+-------------+----------+---------+
| Asian       | Male     |     197 |
+-------------+----------+---------+
| Black       | Female   |      37 |
+-------------+----------+---------+
| Black       | Male     |      37 |
+-------------+----------+---------+
| Caucasian   | Female   |     140 |
+-------------+----------+---------+
| Caucasian   | Male     |     131 |
+-------------+----------+---------+
| Latino      | Female   |     134 |
+-------------+----------+---------+
| Latino      | Male     |     117 |
+-------------+----------+---------+


##### Q. How many employees do each department have? 

In [32]:
query = '''
select 
    Department, 
    count(EEID) as `Count of Employees`
from employees 
group by Department 
order by `Count of Employees` desc;
'''

In [33]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+----------------------+
| Department      |   Count of Employees |
+=================+======================+
| IT              |                  241 |
+-----------------+----------------------+
| Engineering     |                  158 |
+-----------------+----------------------+
| Sales           |                  140 |
+-----------------+----------------------+
| Human Resources |                  125 |
+-----------------+----------------------+
| Finance         |                  120 |
+-----------------+----------------------+
| Marketing       |                  120 |
+-----------------+----------------------+
| Accounting      |                   96 |
+-----------------+----------------------+


##### Q. What is the average annual salary of employees in each department? 

In [34]:
query = '''
select 
    Department, 
    avg(`Annual Salary`) as `Average Salary` 
from employees 
group by Department 
order by `Average Salary` desc;
'''

In [35]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+------------------+
| Department      |   Average Salary |
+=================+==================+
| Marketing       |         129663   |
+-----------------+------------------+
| Accounting      |         123147   |
+-----------------+------------------+
| Finance         |         122803   |
+-----------------+------------------+
| Human Resources |         118058   |
+-----------------+------------------+
| Sales           |         111050   |
+-----------------+------------------+
| Engineering     |         109035   |
+-----------------+------------------+
| IT              |          97790.5 |
+-----------------+------------------+


##### Q. What is the average age of employees in each department?

In [36]:
query = '''
select 
    Department, 
    avg(Age) as 'Average Age'
from employees 
group by Department;
'''

In [37]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+---------------+
| Department      |   Average Age |
+=================+===============+
| IT              |       44.3444 |
+-----------------+---------------+
| Finance         |       45.2917 |
+-----------------+---------------+
| Sales           |       43.6429 |
+-----------------+---------------+
| Accounting      |       43.6563 |
+-----------------+---------------+
| Engineering     |       45.6709 |
+-----------------+---------------+
| Human Resources |       44.456  |
+-----------------+---------------+
| Marketing       |       43.2167 |
+-----------------+---------------+


##### Q. Break down of employee ages. 

In [38]:
query ='''
select 
    Department, 
    sum(case when age between 20 and 29 then 1 else 0 end) as `20-29`,
    sum(case when age between 30 and 39 then 1 else 0 end) as `30-39`,
    sum(case when age between 40 and 49 then 1 else 0 end) as `40-49`,
    sum(case when age between 50 and 59 then 1 else 0 end) as `50-59`,
    sum(case when age between 60 and 69 then 1 else 0 end) as `60-69`, 
    count(EEID) as `Total Count`
from employees
group by Department; 
'''

In [39]:
cur.execute(query)
result = cur.fetchall()
print_result(result)

+-----------------+---------+---------+---------+---------+---------+---------------+
| Department      |   20-29 |   30-39 |   40-49 |   50-59 |   60-69 |   Total Count |
+=================+=========+=========+=========+=========+=========+===============+
| IT              |      29 |      55 |      72 |      66 |      19 |           241 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Finance         |      12 |      33 |      25 |      30 |      20 |           120 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Sales           |      18 |      36 |      41 |      29 |      16 |           140 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Accounting      |      16 |      21 |      25 |      26 |       8 |            96 |
+-----------------+---------+---------+---------+---------+---------+---------------+
| Engineering     |      17 |      30 |      48 |     